# Session 4 — Paragraph-Level Analysis
## Measure 1: Paragraph Semantic Coherence

In this notebook, you will:
- split your text into paragraphs and sentences
- compute **embedding-based coherence** per paragraph using MiniLM
- compare coherence distributions across two books
- connect this to how RAG systems and LLMs assess chunk quality

We again use Lewis Carroll's *Alice* books as examples. Replace the file
paths with your own author/text for your project.


In [ ]:
import re
from typing import List, Tuple
import numpy as np
import matplotlib.pyplot as plt

# You may need to install this once in your environment:
# !pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

def load_book(filepath: str) -> str:
    """Load and lightly clean a book text (Project Gutenberg style)."""
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.read()

    # Very simple Project Gutenberg cleaner (adapt to your own corpus)
    if 'CHAPTER I' in text:
        start = text.find('CHAPTER I')
        text = text[start:]
    elif '*** START OF' in text:
        start = text.find('*** START OF')
        text = text[start + 100:]

    if '*** END OF' in text:
        end = text.find('*** END OF')
        text = text[:end]
    elif 'End of Project Gutenberg' in text:
        end = text.find('End of Project Gutenberg')
        text = text[:end]

    return text.strip()

# Load both Alice books (replace with your own paths)
wonderland_text = load_book('../data/Wonderland.txt')
looking_glass_text = load_book('../data/Looking-Glass.txt')

print(f"Wonderland characters: {len(wonderland_text):,}")
print(f"Looking-Glass characters: {len(looking_glass_text):,}")


In [ ]:
def split_into_paragraphs(text: str, min_words: int = 10) -> List[str]:
    """Split raw text into paragraphs using blank lines as boundaries.
    Filters out very short paragraphs (e.g. chapter titles)."""
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    raw_paras = re.split(r'\n\s*\n+', text)
    paras = []
    for p in raw_paras:
        cleaned = re.sub(r'\s+', ' ', p).strip()
        if not cleaned:
            continue
        if len(cleaned.split()) < min_words:
            continue
        paras.append(cleaned)
    return paras

def sentence_split(paragraph: str) -> List[str]:
    """Very simple sentence splitter based on punctuation."""
    sentences = re.split(r'[.!?]+\s+', paragraph.strip())
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences

def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    if a.ndim > 1:
        a = a.reshape(-1)
    if b.ndim > 1:
        b = b.reshape(-1)
    denom = (np.linalg.norm(a) * np.linalg.norm(b))
    if denom == 0:
        return 0.0
    return float(np.dot(a, b) / denom)

def paragraph_coherence_embeddings(paragraphs: List[str]) -> Tuple[list, list]:
    """Compute embedding-based coherence for each paragraph.

    Steps:
    - Split each paragraph into sentences
    - Compute MiniLM embeddings for all sentences
    - Compute the centroid (mean embedding) for the paragraph
    - Coherence = average cosine similarity of each sentence
      to the centroid embedding
    """
    scores = []
    lengths = []
    for p in paragraphs:
        sents = sentence_split(p)
        if len(sents) < 2:
            continue
        # Encode all sentences in this paragraph
        sent_embs = model.encode(sents)
        centroid = sent_embs.mean(axis=0)
        sims = [cosine_similarity(e, centroid) for e in sent_embs]
        scores.append(sum(sims) / len(sims))
        lengths.append(len(" ".join(sents).split()))
    return scores, lengths

wonderland_paras = split_into_paragraphs(wonderland_text)
looking_glass_paras = split_into_paragraphs(looking_glass_text)

w_scores, w_lengths = paragraph_coherence_embeddings(wonderland_paras)
g_scores, g_lengths = paragraph_coherence_embeddings(looking_glass_paras)

print(f"Wonderland mean coherence (embeddings): {sum(w_scores)/len(w_scores):.3f}")
print(f"Looking-Glass mean coherence (embeddings): {sum(g_scores)/len(g_scores):.3f}")


In [ ]:
# Visualize coherence vs paragraph length
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

axes[0].scatter(w_lengths, w_scores, alpha=0.5)
axes[0].set_title("Wonderland: coherence vs length (MiniLM)")
axes[0].set_xlabel("Paragraph length (tokens)")
axes[0].set_ylabel("Coherence (cosine)")
axes[0].grid(True, alpha=0.3)

axes[1].scatter(g_lengths, g_scores, alpha=0.5)
axes[1].set_title("Looking-Glass: coherence vs length (MiniLM)")
axes[1].set_xlabel("Paragraph length (tokens)")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Compare distributions with histograms
fig, ax = plt.subplots(figsize=(10, 5))

ax.hist(w_scores, bins=20, alpha=0.6, label='Wonderland', density=True)
ax.hist(g_scores, bins=20, alpha=0.6, label='Looking-Glass', density=True)
ax.set_xlabel('Paragraph coherence (cosine similarity to centroid)')
ax.set_ylabel('Density (normalized)')
ax.set_title('Distribution of Paragraph Coherence (MiniLM embeddings)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()


## Memory Cleanup

If you're running low on memory, run this cell to free up RAM by deleting large variables and clearing the model cache.

In [ ]:
import gc

# Delete large variables to free memory
del wonderland_text, looking_glass_text
del wonderland_paras, looking_glass_paras
del w_scores, w_lengths, g_scores, g_lengths

# Clear matplotlib figures
plt.close('all')

# Unload the model from memory
del model

# Force garbage collection
gc.collect()

print("Memory cleaned! Large variables deleted and garbage collected.")